<a href="https://colab.research.google.com/github/elateifsara/MLxDays/blob/master/K_fold_Cross_Validation_with_TF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Taken from : https://www.machinecurve.com/index.php/2020/02/18/how-to-use-k-fold-cross-validation-with-keras/#full-model-code

We call this simple hold-out split, as we simply “hold out” the last 2.000 samples (Chollet, 2017).

It can be a highly effective approach. What’s more, it’s also very inexpensive in terms of the computational power you need. However, it’s also a very naïve approach, as you’ll have to keep these edge cases in mind all the time (Chollet, 2017):

Data representativeness: all datasets, which are essentially samples, must represent the patterns in the population as much as possible. This becomes especially important when you generate samples from a sample (i.e., from your full dataset). For example, if the first part of your dataset has pictures of ice cream, while the latter one only represents espressos, trouble is guaranteed when you generate the split as displayed above. Random shuffling may help you solve these issues.
The arrow of time: if you have a time series dataset, your dataset is likely ordered chronologically. If you’d shuffle randomly, and then perform simple hold-out validation, you’d effectively “[predict] the future given the past” (Chollet, 2017). Such temporal leaks don’t benefit model performance.
Data redundancy: if some samples appear more than once, a simple hold-out split with random shuffling may introduce redundancy between training and testing datasets. That is, identical samples belong to both datasets. This is problematic too, as data used for training thus leaks into the dataset for testing implicitly.
Now, as we can see, while a simple hold-out split based approach can be effective and will be efficient in terms of computational resources, it also requires you to monitor for these edge cases continuously.

**K-fold Cross Validation**

A more expensive and less naïve approach would be to perform K-fold Cross Validation. Here, you set some value for 𝐾 and the dataset is split into 𝐾 partitions of equal size. 𝐾–1 are used for training, while one is used for testing. This process is repeated 𝐾 times, with a different partition used for testing each time.

For example, this would be the scenario for our dataset with 𝐾=5 (i.e., once again the 80/20 split, but then 5 times!).

For each split, the same model is trained, and performance is displayed per fold. For evaluation purposes, you can obviously also average it across all folds. While this produces better estimates, K-fold Cross Validation also increases training cost: in the 𝐾=5 scenario above, the model must be trained for 5 times.

Let’s now extend our viewpoint with a few variations of K-fold Cross Validation 🙂

If you have no computational limitations whatsoever, you might wish to try a special case of K-fold Cross Validation, called Leave One Out Cross Validation (or LOOCV, Khandelwal 2019). LOOCV means 𝐾=𝑁, where 𝑁 is the number of samples in your dataset. As the number of models trained is maximized, the precision of the model performance average is maximized too, but so is the cost of training due to the sheer amount of models that must be trained.

If you have a binary classification problem, you might also wish to take a look at Stratified Cross Validation (Khandelwal, 2019). It extends K-fold Cross Validation by ensuring an equal distribution of the target classes over the splits. This ensures that your classification problem is balanced. It doesn’t work for multiclass classification due to the way that samples are distributed.

Finally, if you have a time series dataset, you might wish to use Time-series Cross Validation (Khandelwal, 2019). [Check here how it works](https://medium.com/datadriveninvestor/k-fold-and-other-cross-validation-techniques-6c03a2563f1e#4a74).

**Creating a Keras model with K-fold Cross Validation**

Now that we understand how K-fold Cross Validation works, it’s time to code an example with the Keras deep learning framework 🙂

Coding it will be a multi-stage process:

* Firstly, we’ll take a look at what we need in order to run our model successfully.
* Then, we take a look at today’s model.
* Subsequently, we add K-fold Cross Validation, train the model instances, and average performance.
* Finally, we output the performance metrics on screen.

In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np

# Model configuration
batch_size = 50
img_width, img_height, img_num_channels = 32, 32, 3
loss_function = sparse_categorical_crossentropy
no_classes = 100
no_epochs = 25
optimizer = Adam()
verbosity = 1
num_folds = 10

# Load CIFAR-10 data
(input_train, target_train), (input_test, target_test) = cifar10.load_data()

# Determine shape of the data
input_shape = (img_width, img_height, img_num_channels)

# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Normalize data
input_train = input_train / 255
input_test = input_test / 255

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
inputs = np.concatenate((input_train, input_test), axis=0)
targets = np.concatenate((target_train, target_test), axis=0)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(no_classes, activation='softmax'))

  # Compile the model
  model.compile(loss=loss_function,
                optimizer=optimizer,
                metrics=['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)

  # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

170500096/170498071 [==============================] - 2s 0us/step
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/25
1080/1080 [==============================] - 36s 3ms/step - loss: 1.8356 - accuracy: 0.3519
Epoch 2/25
1080/1080 [==============================] - 3s 3ms/step - loss: 1.1326 - accuracy: 0.6016
Epoch 3/25
1080/1080 [==============================] - 3s 3ms/step - loss: 0.9549 - accuracy: 0.6659
Epoch 4/25
1080/1080 [==============================] - 3s 3ms/step - loss: 0.8234 - accuracy: 0.7131
Epoch 5/25
1080/1080 [==============================] - 3s 3ms/step - loss: 0.7109 - accuracy: 0.7515
Epoch 6/25
1080/1080 [==============================] - 3s 3ms/step - loss: 0.6256 - accuracy: 0.7816
Epoch 7/25
1080/1080 [==============================] - 3s 3ms/step - loss: 0.5340 - accuracy: 0.8138
Epoch 8/25
1080/1080 [==============================] - 3s 3ms/step - loss: 0.4528 - accuracy: 0.8416
Epoch 9/25
1080/108

Interesting to try without the K-Fold :
- https://www.tensorflow.org/tutorials/structured_data/imbalanced_data [Check METRICS]
- And the following 2 (down) : 
  - https://stackoverflow.com/questions/37657260/how-to-implement-custom-metric-in-keras
  - https://medium.com/@thongonary/how-to-compute-f1-score-for-each-epoch-in-keras-a1acd17715a2


In [15]:
import keras as keras
import numpy as np
from keras.optimizers import SGD
from sklearn.metrics import roc_auc_score

model = keras.models.Sequential()
# ...
sgd = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])


class Metrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        X_val, y_val = self.validation_data[0], self.validation_data[1]
        y_predict = np.asarray(model.predict(X_val))

        y_val = np.argmax(y_val, axis=1)
        y_predict = np.argmax(y_predict, axis=1)

        self._data.append({
            'val_rocauc': roc_auc_score(y_val, y_predict),
        })
        return

    def get_data(self):
        return self._data

metrics = Metrics()
#history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[metrics])
#metrics.get_data()

In [22]:
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

class Metrics(Callback):
  def on_train_begin(self, logs={}):
    self.val_f1s = []
    self.val_recalls = []
    self.val_precisions = []
 
  def on_epoch_end(self, epoch, logs={}):
    val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
    val_targ = self.model.validation_data[1]
    _val_f1 = f1_score(val_targ, val_predict)
    _val_recall = recall_score(val_targ, val_predict)
    _val_precision = precision_score(val_targ, val_predict)
    self.val_f1s.append(_val_f1)
    self.val_recalls.append(_val_recall)
    self.val_precisions.append(_val_precision)
    #print("— val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall)
    return
  
  def get_data(self):
    return self.val_f1s, self.val_recalls, self.val_precisions

metrics = Metrics()

# Please call these in an no K-fold thing

# Coming back to K-Fold

In [21]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np

import sklearn
from sklearn.preprocessing import binarize
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, plot_precision_recall_curve, f1_score, confusion_matrix

# Model configuration
batch_size = 50
img_width, img_height, img_num_channels = 32, 32, 3
loss_function = sparse_categorical_crossentropy
no_classes = 100
no_epochs = 2#25
optimizer = Adam()
verbosity = 1
num_folds = 5

# Load CIFAR-10 data
(input_train, target_train), (input_test, target_test) = cifar10.load_data()

# Determine shape of the data
input_shape = (img_width, img_height, img_num_channels)

# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Normalize data
input_train = input_train / 255
input_test = input_test / 255

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
predictions = []
ground_truths = []

# Merge inputs and targets
inputs = np.concatenate((input_train, input_test), axis=0)
targets = np.concatenate((target_train, target_test), axis=0)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(no_classes, activation='softmax'))


  # Compile the model
  model.compile(loss=loss_function,
                optimizer=optimizer,
                metrics=['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)

  # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  prediction = model.predict(inputs[test])
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  predictions.append(prediction)
  ground_truths.append(targets[test])
  
  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
#print(f'> F1-score: {np.mean(f1score_per_fold)} (+- {np.std(f1score_per_fold)})')
#print(f'> F1-score: {np.mean(metrics.val_f1s)} (+- {np.std(metrics.val_f1s)})')
#print(f'> Precision: {np.mean(metrics.val_precisions)} (+- {np.std(metrics.val_precisions)})')
#print(f'> Precision: {np.mean(metrics.val_recalls)} (+- {np.std(metrics.val_recalls)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/2
960/960 [==============================] - 4s 3ms/step - loss: 1.8786 - accuracy: 0.3401
Epoch 2/2
960/960 [==============================] - 3s 3ms/step - loss: 1.1876 - accuracy: 0.5798
Score for fold 1: loss of 1.0698779821395874; accuracy of 62.5249981880188%
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/2
960/960 [==============================] - 3s 3ms/step - loss: 1.8136 - accuracy: 0.3442
Epoch 2/2
960/960 [==============================] - 3s 3ms/step - loss: 1.1676 - accuracy: 0.5805
Score for fold 2: loss of 1.0498671531677246; accuracy of 62.74999976158142%
------------------------------------------------------------------------
Training for fold 3 ...
Epoch 1/2
960/960 [==============================] - 3s 3ms/step - loss: 1.8373 - accuracy: 0.3376
Epoch 2/2
960/960 [==============================] - 3s 3ms/s

In [23]:
len(predictions), len(ground_truths)

(5, 5)

In [36]:
for elt1, elt2 in zip([1.0, 0.0, 1.0, 1.0, 0.0], [1.0, 0.0, 1.0, 1.0, 0.0]):
  #print(np.rint(np.max(elt1,axis=1)))
  precision, recall, _ = precision_recall_curve([elt2], [elt1])
  prec_per_fold.append(precision)


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_ranking.py:677: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


In [ ]:
sens_per_fold = []
spec_per_fold = []
prec_per_fold = []
auc_per_fold = []
f1score_per_fold = []

## ADD CALCULATION OF AUC + F1-SCORE + ROC + PRECISION + RECALL + SPECIFICTY + SENSITIVTY (and store these in df and save it + the model)
  ## sensitivity == recall
def calculate_metrics(t_y, t_p):
  precision, recall, _ = precision_recall_curve(t_y,p_y)
  prec_per_fold.append(precision)
  sens_per_fold.append(recall)

  fpr, tpr, _ = roc_curve(t_y,p_y)
  auc = auc(fpr, tpr)
  auc_per_fold.append(auc)

  tn, fp, fn, tp = sklearn.metrics.confusion_matrix(t_y,p_y).ravel() 
  spec = tn/(tn+fp)
  spec_per_fold.append(spec)

  f1_score = 2*(precision*recall)/(precision+recall)
  f1score_per_fold.append(f1_score)
  return sens_per_fold, spec_per_fold, prec_per_fold, auc_per_fold, f1score_per_fold